In [4]:
import networkx as nx
import pandas as pd
import random
import torch
import pickle
from sklearn.model_selection import train_test_split
import numpy as np

In [5]:
### import parquet file
unsw_w_st = pd.read_parquet("C:\\Users\\asus\\Documents\\nids-pcap-dataset\\unsw_parquet_used_dataset\\labeled_wo_start_time_w_dup[port].parquet")

In [6]:
unsw_w_st.head(5)

,source_ip,destination_ip,source_port,destination_port,info_message,attack_category,is_malware,source_ip_info,source_port_info,dest_ip_info,dest_port_info,count_benign,count_malware
index,,,,,,,,,,,,,
1,175.45.176.1,149.171.126.18,4657,80,GET /oKmwKoVbq HTTP/1.1,NaN,0,175.45.176.1 GET /oKmwKoVbq HTTP/1.1,4657 GET /oKmwKoVbq HTTP/1.1,149.171.126.18 GET /oKmwKoVbq HTTP/1.1,80 GET /oKmwKoVbq HTTP/1.1,1,0
2,175.45.176.3,149.171.126.18,32473,80,GET /level/15/exec/-/buffers/assigned/dump HTT...,NaN,1,175.45.176.3 GET /level/15/exec/-/buffers/assi...,32473 GET /level/15/exec/-/buffers/assigned/du...,149.171.126.18 GET /level/15/exec/-/buffers/as...,80 GET /level/15/exec/-/buffers/assigned/dump ...,1,7
6,175.45.176.0,149.171.126.17,49194,80,GET eLWfxXSPkc HTTP/1.1,NaN,0,175.45.176.0 GET eLWfxXSPkc HTTP/1.1,49194 GET eLWfxXSPkc HTTP/1.1,149.171.126.17 GET eLWfxXSPkc HTTP/1.1,80 GET eLWfxXSPkc HTTP/1.1,1,0
9,175.45.176.1,149.171.126.14,51435,80,GET / HTTP/1.1,NaN,0,175.45.176.1 GET / HTTP/1.1,51435 GET / HTTP/1.1,149.171.126.14 GET / HTTP/1.1,80 GET / HTTP/1.1,46862,1313
10,175.45.176.1,149.171.126.19,64694,80,GET /scripts/cbag/ag.exe?page=FileDownload&id=...,NaN,0,175.45.176.1 GET /scripts/cbag/ag.exe?page=Fil...,64694 GET /scripts/cbag/ag.exe?page=FileDownlo...,149.171.126.19 GET /scripts/cbag/ag.exe?page=F...,80 GET /scripts/cbag/ag.exe?page=FileDownload&...,1,0


In [7]:
unsw_w_st.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125325 entries, 1 to 490022
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   source_ip         125325 non-null  object  
 1   destination_ip    125325 non-null  object  
 2   source_port       125325 non-null  object  
 3   destination_port  125325 non-null  object  
 4   info_message      125325 non-null  object  
 5   attack_category   15801 non-null   category
 6   is_malware        125325 non-null  int64   
 7   source_ip_info    125325 non-null  object  
 8   source_port_info  125325 non-null  object  
 9   dest_ip_info      125325 non-null  object  
 10  dest_port_info    125325 non-null  object  
 11  count_benign      125325 non-null  int64   
 12  count_malware     125325 non-null  int64   
dtypes: category(1), int64(3), object(9)
memory usage: 12.5+ MB


In [8]:
unsw_w_st['source_port'] = unsw_w_st.source_port.astype('int32')
unsw_w_st['destination_port']= unsw_w_st.destination_port.astype('int32')

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
unsw_w_st['sip_ohe'] = label_encoder.fit_transform(unsw_w_st.source_ip.values)

In [11]:
unsw_w_st['lensip'] = unsw_w_st.info_message.str.len()

In [12]:
unsw_w_st.loc[unsw_w_st['is_malware'] == 0, 'destination_port'].value_counts().sort_values()

destination_port
80    110274
Name: count, dtype: int64

### Split Training Testing

In [13]:
train_df, test_df = train_test_split(unsw_w_st, test_size=0.3, random_state=0)

In [14]:
train_df.is_malware.value_counts()

is_malware
0    77148
1    10579
Name: count, dtype: int64

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87727 entries, 202866 to 255741
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   source_ip         87727 non-null  object  
 1   destination_ip    87727 non-null  object  
 2   source_port       87727 non-null  int32   
 3   destination_port  87727 non-null  int32   
 4   info_message      87727 non-null  object  
 5   attack_category   11119 non-null  category
 6   is_malware        87727 non-null  int64   
 7   source_ip_info    87727 non-null  object  
 8   source_port_info  87727 non-null  object  
 9   dest_ip_info      87727 non-null  object  
 10  dest_port_info    87727 non-null  object  
 11  count_benign      87727 non-null  int64   
 12  count_malware     87727 non-null  int64   
 13  sip_ohe           87727 non-null  int32   
 14  lensip            87727 non-null  int64   
dtypes: category(1), int32(3), int64(4), object(7)
memory usage: 9.1+ MB


In [16]:
# value_counts = train_df.is_malware.value_counts()
# if value_counts[1] > 102:
#     df_to_lower = train_df[train_df['is_malware'] == 1].sample(n=104, random_state=13)
#     train_df = pd.concat([train_df[train_df['is_malware'] == 0], df_to_lower])

In [17]:
# nD_train_df = train_df.drop_duplicates(subset=['dest_port_info'])
label_train = train_df['is_malware'].to_numpy()
label_train_tensor = torch.tensor(label_train, dtype=torch.float)
value_counts = np.unique(label_train, return_counts=True)
value_counts

(array([0, 1], dtype=int64), array([77148, 10579], dtype=int64))

In [31]:
nD_train =  train_df.drop_duplicates(subset=['dest_port_info'])

In [32]:
nD_train.is_malware.value_counts()

is_malware
1    5965
0    1674
Name: count, dtype: int64

In [18]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37598 entries, 191480 to 459269
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   source_ip         37598 non-null  object  
 1   destination_ip    37598 non-null  object  
 2   source_port       37598 non-null  int32   
 3   destination_port  37598 non-null  int32   
 4   info_message      37598 non-null  object  
 5   attack_category   4682 non-null   category
 6   is_malware        37598 non-null  int64   
 7   source_ip_info    37598 non-null  object  
 8   source_port_info  37598 non-null  object  
 9   dest_ip_info      37598 non-null  object  
 10  dest_port_info    37598 non-null  object  
 11  count_benign      37598 non-null  int64   
 12  count_malware     37598 non-null  int64   
 13  sip_ohe           37598 non-null  int32   
 14  lensip            37598 non-null  int64   
dtypes: category(1), int32(3), int64(4), object(7)
memory usage: 3.9+ MB


In [19]:
# nD_test_df = test_df.drop_duplicates(subset=['dest_port_info'])
label_test = test_df['is_malware'].to_numpy()
value_counts = np.unique(label_test, return_counts=True)
label_test_tensor = torch.tensor(label_test, dtype=torch.float)
value_counts

(array([0, 1], dtype=int64), array([33126,  4472], dtype=int64))

In [20]:
test_df.is_malware.value_counts()

is_malware
0    33126
1     4472
Name: count, dtype: int64

In [33]:
nD_test_df = test_df.drop_duplicates(subset=['dest_port_info'])
nD_test_df.is_malware.value_counts()

is_malware
1    3056
0     789
Name: count, dtype: int64

#### training graph

In [37]:
nD_graph_train = nx.Graph()
node_features = []
attr = []
labels = []

for dest_port_info in train_df["dest_port_info"]:
    nD_graph_train.add_node(dest_port_info)
    info_message = train_df[train_df["dest_port_info"] == dest_port_info]["info_message"].iloc[0]
    node_features.append([float(len(info_message))])
    
for (source_ip), group in train_df.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["dest_port_info"]
        to_node = group.iloc[i+1]["dest_port_info"]
        if nD_graph_train.has_edge(from_node, to_node):
            nD_graph_train[from_node][to_node]["weight"] += 1
        else:
            nD_graph_train.add_edge(from_node, to_node, weight=1)

In [38]:
node_features_tensor = torch.tensor(node_features)
node_features_tensor = node_features_tensor.cuda()

In [39]:
node_features_tensor

tensor([[ 15.],
        [ 28.],
        [ 28.],
        ...,
        [168.],
        [ 28.],
        [ 28.]], device='cuda:0')

In [41]:
## print num of nodes
print("Number of nodes:", nD_graph_train.number_of_nodes(), "\n")

Number of nodes: 7639 



In [36]:
label_train = train_df['is_malware'].to_numpy()
label_train_tensor = torch.tensor(label_train, dtype=torch.float)
value_counts = np.unique(label_train, return_counts=True)
value_counts

(array([0, 1], dtype=int64), array([77148, 10579], dtype=int64))

In [24]:
isolated_nodes = list(nx.isolates(nD_graph_train))

print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



#### testing graph

In [20]:
nD_graph_test = nx.Graph()
node_features_test = []
labels_test = []
ground_truth = []

for dest_port_info in test_df["dest_port_info"].unique():
    nD_graph_test.add_node(dest_port_info)
    info_message = test_df[test_df["dest_port_info"] == dest_port_info]["info_message"].iloc[0]
    ground_truth = test_df[test_df["dest_port_info"] == dest_port_info]["is_malware"].iloc[0]
    # print(info_message)
    node_features_test.append([float(len(info_message))])

for (source_ip), group in test_df.groupby(["source_ip"]):
    for i in range(len(group) - 1):
        from_node = group.iloc[i]["dest_port_info"]
        to_node = group.iloc[i+1]["dest_port_info"]
        if nD_graph_test.has_edge(from_node, to_node):
            nD_graph_test[from_node][to_node]["weight"] += 1
        else:
            nD_graph_test.add_edge(from_node, to_node, weight=1)

In [21]:
# ground_truth = ground_truth.to_numpy()
# ground_truth = torch.tensor(ground_truth, dtype=torch.float)

In [22]:
node_features_tensor_test = torch.tensor(node_features_test)
node_features_tensor_test = node_features_tensor_test.cuda()

In [23]:
node_features_tensor_test

tensor([[  12.],
        [  15.],
        [  28.],
        ...,
        [  39.],
        [1047.],
        [  67.]], device='cuda:0')

In [24]:
isolated_nodes = list(nx.isolates(nD_graph_test))

print("Isolated nodes:", len(isolated_nodes), "\n")

Isolated nodes: 0 



### GAD

In [26]:
import torch
from torch_geometric.utils.convert import from_networkx
from pygod.detector import DOMINANT, OCGNN, GUIDE, GAE, GAAN, AnomalyDAE, CONAD
from pygod.metric import eval_average_precision, eval_roc_auc, eval_f1, eval_precision_at_k, eval_recall_at_k
from pygod.generator import gen_contextual_outlier, gen_structural_outlier
import pickle
device = torch.device('cuda')

In [28]:
pyG_train = from_networkx(nD_graph_train)
pyG_train = pyG_train
pyG_train.x = node_features_tensor

# pyG_test = from_networkx(nD_graph_test)
# pyG_test = pyG_test
# pyG_test.x = node_features_tensor_test

#### DOMINANT

In [30]:
dominant_model = DOMINANT(gpu=0, weight=0.02, num_layers=144, hid_dim=64, contamination=0.12, lr=0.001, verbose=3, epoch=100)

In [34]:
dominant_compile = dominant_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 5.0731 | 

ValueError: Found input variables with inconsistent numbers of samples: [87727, 7639]

In [29]:
dominant_ip_pred_res, dominant_ip_score_res, dominant_ip_prob_res, dominant_ip_conf_res = dominant_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(dominant_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0011 | AUC 0.6186 | Recall 0.8367 | Precision 0.8367 | AP 0.8377 | F1 0.8367 | Time 0.65
tensor([0, 1]) tensor([3425,  420])


#### OCGNN

In [32]:
ocgnn_model = OCGNN(hid_dim=64, num_layers=64, weight_decay=0.02, 
                    contamination=0.12, lr=0.001, epoch=100, gpu=-1, batch_size=0, num_neigh=-1, 
                    beta=0.5, warmup=2, eps=0.001, verbose=3)

In [33]:
ocgnn_compile = ocgnn_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 0.0000 | AUC 0.5783 | Recall 0.7992 | Precision 0.7992 | AP 0.8312 | F1 0.7992 | Time 2.20
Epoch 0001: Loss 0.0003 | AUC 0.6027 | Recall 0.8052 | Precision 0.8052 | AP 0.8465 | F1 0.8051 | Time 2.03
Epoch 0002: Loss 0.0002 | AUC 0.5915 | Recall 0.7951 | Precision 0.7951 | AP 0.8459 | F1 0.7951 | Time 1.98
Epoch 0003: Loss 0.0001 | AUC 0.5497 | Recall 0.7821 | Precision 0.7821 | AP 0.8335 | F1 0.7821 | Time 2.02
Epoch 0004: Loss 0.0001 | AUC 0.5164 | Recall 0.7816 | Precision 0.7816 | AP 0.8097 | F1 0.7816 | Time 1.98
Epoch 0005: Loss 0.0001 | AUC 0.5130 | Recall 0.7807 | Precision 0.7807 | AP 0.8037 | F1 0.7807 | Time 1.96
Epoch 0006: Loss 0.0001 | AUC 0.5073 | Recall 0.7837 | Precision 0.7837 | AP 0.7874 | F1 0.7837 | Time 1.98
Epoch 0007: Loss 0.0001 | AUC 0.4945 | Recall 0.7826 | Precision 0.7826 | AP 0.7592 | F1 0.7826 | Time 2.00
Epoch 0008: Loss 0.0001 | AUC 0.4816 | Recall 0.7854 | Precision 0.7854 | AP 0.7435 | F1 0.7854 | Time 1.99
Epoch 0009: Loss 0.0001 | AU

In [34]:
ocgnn_ip_pred_res, ocgnn_ip_score_res, ocgnn_ip_prob_res, ocgnn_ip_conf_res = ocgnn_compile.predict(data=pyG_test, label=label_test_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(ocgnn_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0000 | AUC 0.3317 | Recall 0.7683 | Precision 0.7683 | AP 0.6935 | F1 0.7681 | Time 0.39
tensor([0, 1]) tensor([  99, 3746])


In [35]:
f1_score_ip = eval_f1(label_test_tensor, ocgnn_ip_pred_res)
print(f1_score_ip)

0.8709203175536608


#### GUIDE

In [36]:
guide_model = GUIDE(num_layers=4, hid_a=16, hid_s=16, weight_decay=0.05,  
                    contamination=0.12, lr=0.001, epoch=100, gpu=-1, dropout=0.5,
                    verbose=3)

In [37]:
print(guide_model)

GUIDE(act=<function relu at 0x000002F22EE0A8B0>, alpha=0.5, backbone=None,
      batch_size=0, cache_dir=None, compile_model=False,
      contamination=0.12, dropout=0.5, epoch=100, gpu=None,
      graphlet_size=4, hid_a=None, hid_s=None, lr=0.001, num_layers=4,
      num_neigh=[-1, -1, -1, -1], save_emb=False, selected_motif=True,
      verbose=3, weight_decay=0.05)


In [38]:
guide_compile = guide_model.fit(pyG_train)

KeyboardInterrupt: 

In [ ]:
guide_ip_pred_res, guide_ip_score_res, guide_ip_prob_res, guide_ip_conf_res = guide_compile.predict(data=pyG_test, label=label_test_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(guide_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.8848 | AUC 1.0000 | Recall 1.0000 | Precision 1.0000 | AP 1.0000 | F1 1.0000 | Time 0.23
tensor([0, 1]) tensor([4440, 2583])


#### GAE

In [81]:
gae_model = GAE(hid_dim=64, num_layers=128, weight_decay=0.2, dropout=0.5,
                contamination=0.12, lr=0.001, epoch=100, gpu=-1,
                num_neigh=-1, verbose=3)

In [82]:
gae_compile = gae_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 376054.5625 | AUC 0.5752 | Recall 0.7832 | Precision 0.7832 | AP 0.8389 | F1 0.7743 | Time 4.25
Epoch 0001: Loss 376052.3750 | AUC 0.5735 | Recall 0.7812 | Precision 0.7812 | AP 0.8391 | F1 0.7812 | Time 3.87
Epoch 0002: Loss 376049.5625 | AUC 0.5733 | Recall 0.7819 | Precision 0.7819 | AP 0.8390 | F1 0.7819 | Time 4.04
Epoch 0003: Loss 376046.2812 | AUC 0.5730 | Recall 0.7816 | Precision 0.7816 | AP 0.8388 | F1 0.7816 | Time 3.97
Epoch 0004: Loss 376042.8125 | AUC 0.5732 | Recall 0.7807 | Precision 0.7807 | AP 0.8390 | F1 0.7807 | Time 3.97
Epoch 0005: Loss 376039.2500 | AUC 0.5732 | Recall 0.7817 | Precision 0.7817 | AP 0.8389 | F1 0.7817 | Time 4.00
Epoch 0006: Loss 376034.2812 | AUC 0.5732 | Recall 0.7817 | Precision 0.7817 | AP 0.8391 | F1 0.7817 | Time 3.93


KeyboardInterrupt: 

In [80]:
gae_ip_pred_res, gae_ip_score_res, gae_ip_prob_res, gae_ip_conf_res = gae_compile.predict(data=pyG_test, label=label_test_tensor, return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(gae_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 85.4987 | AUC 0.5860 | Recall 0.7948 | Precision 0.7948 | AP 0.8682 | F1 0.7948 | Time 0.91
tensor([0, 1]) tensor([3554,  291])


#### GAAN

In [88]:
gaan_model = GAAN(noise_dim=8, hid_dim=8, num_layers=8, dropout=0.0, 
                  weight_decay=0.02, contamination=0.12, lr=0.004, 
                  epoch=100, gpu=-1, batch_size=0, num_neigh=0, 
                  weight=0.5, verbose=3)

In [89]:
gaan_compile = gaan_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss G nan | Loss D nan | AUC 0.5810 | Recall 0.7866 | Precision 0.7866 | AP 0.8412 | F1 0.7866 | Time 2.25
Epoch 0001: Loss G nan | Loss D nan | AUC 0.5805 | Recall 0.7864 | Precision 0.7864 | AP 0.8410 | F1 0.7864 | Time 1.85
Epoch 0002: Loss G nan | Loss D nan | AUC 0.5805 | Recall 0.7858 | Precision 0.7858 | AP 0.8412 | F1 0.7858 | Time 1.86
Epoch 0003: Loss G nan | Loss D nan | AUC 0.5810 | Recall 0.7856 | Precision 0.7856 | AP 0.8413 | F1 0.7856 | Time 1.89
Epoch 0004: Loss G nan | Loss D nan | AUC 0.5808 | Recall 0.7859 | Precision 0.7859 | AP 0.8411 | F1 0.7859 | Time 1.93
Epoch 0005: Loss G nan | Loss D nan | AUC 0.5809 | Recall 0.7863 | Precision 0.7863 | AP 0.8411 | F1 0.7863 | Time 2.04
Epoch 0006: Loss G nan | Loss D nan | AUC 0.5804 | Recall 0.7866 | Precision 0.7866 | AP 0.8412 | F1 0.7866 | Time 1.98
Epoch 0007: Loss G nan | Loss D nan | AUC 0.5806 | Recall 0.7864 | Precision 0.7864 | AP 0.8413 | F1 0.7864 | Time 1.84
Epoch 0008: Loss G nan | Loss D nan | AU

In [90]:
gaan_ip_pred_res, gaan_ip_score_res, gaan_ip_prob_res, gaan_ip_conf_res = gaan_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(gaan_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss nan | AUC 0.5099 | Recall 0.7938 | Precision 0.7938 | AP 0.8103 | F1 0.7938 | Time 0.48
tensor([0, 1]) tensor([3358,  487])


#### CONAD

In [93]:
conad_mode = CONAD(hid_dim=64, num_layers=64, dropout=0.0, weight_decay=0.0, contamination=0.12, lr=0.001, 
      epoch=100, gpu=-1, num_neigh=-1, weight=0.5, eta=0.5, margin=0.5, verbose=3)

In [94]:
conda_compile = conad_mode.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 43.4965 | AUC 0.5820 | Recall 0.7864 | Precision 0.7864 | AP 0.8409 | F1 0.7851 | Time 6.70
Epoch 0001: Loss 43.4961 | AUC 0.5821 | Recall 0.7864 | Precision 0.7864 | AP 0.8415 | F1 0.7864 | Time 6.47
Epoch 0002: Loss 43.4919 | AUC 0.5820 | Recall 0.7861 | Precision 0.7861 | AP 0.8418 | F1 0.7861 | Time 6.50
Epoch 0003: Loss 43.4889 | AUC 0.5820 | Recall 0.7863 | Precision 0.7863 | AP 0.8417 | F1 0.7863 | Time 6.87
Epoch 0004: Loss 43.4835 | AUC 0.5820 | Recall 0.7863 | Precision 0.7863 | AP 0.8417 | F1 0.7863 | Time 6.69
Epoch 0005: Loss 43.4810 | AUC 0.5815 | Recall 0.7861 | Precision 0.7861 | AP 0.8415 | F1 0.7861 | Time 6.38
Epoch 0006: Loss 43.4760 | AUC 0.5812 | Recall 0.7859 | Precision 0.7859 | AP 0.8414 | F1 0.7859 | Time 6.88
Epoch 0007: Loss 43.4690 | AUC 0.5810 | Recall 0.7861 | Precision 0.7861 | AP 0.8413 | F1 0.7861 | Time 6.38
Epoch 0008: Loss 43.4593 | AUC 0.5809 | Recall 0.7861 | Precision 0.7861 | AP 0.8413 | F1 0.7861 | Time 6.39
Epoch 0009: Loss 43

In [96]:
conad_ip_pred_res, conad_ip_score_res, conad_ip_prob_res, conad_ip_conf_res = conda_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(conad_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0183 | AUC 0.5953 | Recall 0.8226 | Precision 0.8226 | AP 0.8325 | F1 0.8226 | Time 0.86
tensor([0, 1]) tensor([3354,  491])


#### AnomalyDAE

In [101]:
anomalydae_model = AnomalyDAE(emb_dim=128, hid_dim=128, num_layers=68, weight_decay=0.02, 
                            backbone=None, alpha=0.5, theta=1.0, eta=1.0, 
                            contamination=0.1, lr=0.004, epoch=100, gpu=-1, 
                            verbose=3)

In [102]:
anomalydae_compile = anomalydae_model.fit(pyG_train, label_train_tensor)

Epoch 0000: Loss 322.5231 | AUC 0.5455 | Recall 0.7839 | Precision 0.7839 | AP 0.8296 | F1 0.7839 | Time 2.27
Epoch 0001: Loss 786615.7500 | AUC 0.5049 | Recall 0.7975 | Precision 0.7975 | AP 0.7971 | F1 0.7975 | Time 2.19
Epoch 0002: Loss 180216.4688 | AUC 0.5083 | Recall 0.7953 | Precision 0.7953 | AP 0.7985 | F1 0.7953 | Time 2.14
Epoch 0003: Loss 288449.0938 | AUC 0.5111 | Recall 0.7940 | Precision 0.7940 | AP 0.8001 | F1 0.7940 | Time 2.21
Epoch 0004: Loss 85409.9531 | AUC 0.5130 | Recall 0.7928 | Precision 0.7928 | AP 0.8010 | F1 0.7928 | Time 2.20
Epoch 0005: Loss 260072.4219 | AUC 0.5112 | Recall 0.7921 | Precision 0.7921 | AP 0.8001 | F1 0.7921 | Time 2.14
Epoch 0006: Loss 201477.9219 | AUC 0.5113 | Recall 0.7920 | Precision 0.7920 | AP 0.8002 | F1 0.7920 | Time 2.17
Epoch 0007: Loss 43496.6055 | AUC 0.5105 | Recall 0.7908 | Precision 0.7908 | AP 0.7997 | F1 0.7908 | Time 2.31
Epoch 0008: Loss 108959.7188 | AUC 0.5137 | Recall 0.7909 | Precision 0.7909 | AP 0.8015 | F1 0.7909 

In [103]:
anomalydae_ip_pred_res, anomalydae_ip_score_res, anomalydae_ip_prob_res, anomalydae_ip_conf_res = conda_compile.predict(data=pyG_test, label = label_test_tensor,return_pred=True, return_score=True, return_prob=True, prob_method='linear', return_conf=True)
unique_values, counts = torch.unique(anomalydae_ip_pred_res, return_counts=True)
print(unique_values, counts)

Test: Loss 0.0183 | AUC 0.5953 | Recall 0.8226 | Precision 0.8226 | AP 0.8325 | F1 0.8226 | Time 0.86
tensor([0, 1]) tensor([3354,  491])
